In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from pandas_profiling import ProfileReport
import plotly.express as px
from scipy import stats
import warnings
warnings.filterwarnings("ignore")


In [89]:
df=pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [90]:
df=df.drop(['region'],axis=1)

In [91]:
data = ff.create_table(df.head())
data

# EDA WITH SWEETVIZ AND DTALE

In [92]:
import sweetviz as sv

In [93]:
report=sv.analyze(df)

In [94]:
# view and save
report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [95]:
pip install sweetviz datale pandas_profiling

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement datale (from versions: none)
ERROR: No matching distribution found for datale


In [96]:
import dtale

In [ ]:
d=dtale.show(df)

In [ ]:
# view result
d

In [ ]:
# checking for missing values
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

categorical variables

In [ ]:
# DISCLAIMER: 'df' refers to the data you passed in when calling 'dtale.show'

import numpy as np
import pandas as pd
import plotly.graph_objs as go

if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)

# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

s = df[~pd.isnull(df['age'])][['age']]
chart, labels = np.histogram(s, bins=20)
import scipy.stats as sts
kde = sts.gaussian_kde(s['age'])
kde_data = kde.pdf(np.linspace(labels.min(), labels.max()))
# main statistics
stats = df['age'].describe().to_frame().T
charts = [
	go.Bar(x=labels[1:], y=chart, name='age distrubution'),
	go.Scatter(
		x=list(range(len(kde_data))), y=kde_data, name='KDE',		yaxis='y2', xaxis='x2',		line={'shape': 'spline', 'smoothing': 0.3}, mode='lines'
	)
]
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'group',
    'legend': {'orientation': 'h'},
    'title': {'text': 'age Histogram (bins: 20) w/ KDE'},
    'xaxis2': {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
    'yaxis': {'side': 'left', 'title': {'text': 'Frequency'}},
    'yaxis2': {'overlaying': 'y', 'side': 'right', 'title': {'text': 'KDE'}}
}))
figure.show()

as we can see from  the histogram above our dataset mostlty consits of early 20s  and the other age distrubution  is normaly distrubuted 

In [ ]:
# make a list of the numerical variables first
numerical = [var for var in df.columns if df[var].dtype!='O']
print(numerical)

In [ ]:
numerical_continous=df[['age','charges','bmi']]
numerical_continous.shape

In [ ]:
numerical_discrete=df['children']
numerical_discrete.head()

# checking and removing outliers in numerical_continous data by z-score

In [ ]:
plt.boxplot([df.age,df.bmi] , labels = ['age' , 'bmi'])
plt.show()

In [ ]:
from scipy import stats
z=np.abs(stats.zscore(df.bmi))
print(z)

In [ ]:
df.shape

In [ ]:
threshold=2
print(np.where(z>3))

In [ ]:
df_clean=df[(z< 2)]
print(df_clean)


In [ ]:
plt.boxplot([df_clean.age, df_clean.bmi] , labels = ['age' , 'bmi'])
plt.show()

THE PLOT ABOVE SHOWS NO MORE OUTLIERS, IT SHOWS THAT WITH THRESHOLD OF 2, MEANING IF WE MAKE OUR OUTPUT TO BE BETWEEN -2 AND 2, WE ELIMANATE OUTLIERS.

In [ ]:
# find categorical variables
categorical = [var for var in df_clean.columns if df_clean[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))

In [ ]:

for var in df_clean.columns :
      if df_clean[var].dtype=='O':
             print(var)
             #print('There are {} categorical variables'.format(len(categorical)))

In [ ]:
categorical=df_clean[['sex','smoker']]
categorical

In [ ]:
categorical_discrete=df_clean['children']
categorical_discrete

In [ ]:
df_clean

# encoding categorical features

In [ ]:
#separating target variable from features
# creating x(all feature columns except target)
x=df_clean.drop("charges",axis=1)

# creating y (the output column)
y=df_clean['charges']

In [ ]:
x

In [ ]:
y

In [ ]:
#Creating input data from dataframe df on variable Positive Skewness with input values ranging from 1 to 130
inp_array = y 
print ("Input array : ", inp_array)

In [ ]:
#Applying log10 transformation with output values ranging from 0 to 2+
y = np.log10(inp_array) 
print ("out_array : ", y)

In [ ]:
x=pd.get_dummies(x,columns=['smoker','sex'])
x.head()

# feature selection

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df_clean.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["charges"])
cor_target

In [ ]:
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
relevant_features

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
x_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
y

we do log transfomation on target variable to make the values have a normal distrubution and  remove outliers

# Random Forest Regressor

In [ ]:
>>> from sklearn.ensemble import RandomForestRegressor


In [ ]:
model=RandomForestRegressor(n_jobs=-1,
                              random_state=50)
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

# LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# regression_model = LinearRegression()
regression_model.fit(x_train, y_train)

In [ ]:
regression_model.score(x_test,y_test)

# gradient Boosting regressor

In [ ]:
>>> import numpy as np
>>> from sklearn.metrics import mean_squared_error
>>> from sklearn.datasets import make_friedman1
>>> from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
>>> x, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)

In [ ]:
>>> est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
...     max_depth=1, random_state=0, loss='ls').fit(x_train, y_train)
>>> mean_squared_error(y_test, est.predict(x_test))

# RandomizedSearchCV

In [ ]:

%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": np.arange(100, 1000, 100),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"]}

# Instantiate RandomizedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-3,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=5,
                              cv=4,
                              verbose=True)

# Fit the RandomizedSearchCV model
rs_model.fit(x_train, y_train)

In [ ]:
rs_model.score(x_test,y_test)

# Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor



In [ ]:
model=RandomForestRegressor(n_jobs=-8,
                              random_state=80)
model.fit(x_train,y_train)

In [ ]:
 y_pred = model.predict(x_test)

# MEAN ABSOLUTE ERROR

In [ ]:
#Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)




# R² Score

In [ ]:

from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

# MEAN SQUARED ERROR

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)
